In [ ]:
!git clone https://github.com/saberonjr/PlantDoc-Object-Detection-Dataset.git

Cloning into 'PlantDoc-Object-Detection-Dataset'...
remote: Enumerating objects: 5196, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 5196 (delta 2), reused 5 (delta 2), pack-reused 5190
Receiving objects: 100% (5196/5196), 941.89 MiB | 40.16 MiB/s, done.
Resolving deltas: 100% (2330/2330), done.
Updating files: 100% (5200/5200), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


# **Zip the sample files**

In [ ]:
import shutil

def zip_folder(folder_path, zip_path):
    shutil.make_archive(zip_path, 'zip', folder_path)

# Specify the folder path to be zipped
folder_path = '/content/PlantDoc-Object-Detection-Dataset'

# Specify the path for the zip file to be created
zip_path = '/content/PlantDoc-Object-Detection-Dataset.zip'

# Call the function to zip the folder
zip_folder(folder_path, zip_path)

# **Copy the zip file in google drive**

In [ ]:
from google.colab import drive
import shutil

# Mount Google Drive
drive.mount('/content/drive')

# Define paths
zip_file_path_colab = '/content/PlantDoc-Object-Detection-Dataset.zip.zip'
destination_folder_drive = '/content/drive/My Drive/'

# Copy the zip file to Google Drive
shutil.copy(zip_file_path_colab, destination_folder_drive)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/My Drive/PlantDoc-Object-Detection-Dataset.zip.zip'

# **Covert xml to yolo format**

In [ ]:
import os
import xml.etree.ElementTree as ET

def convert_xml_to_yolo(xml_file, image_width, image_height):
    print(f"Converting XML file: {xml_file}")

    root = ET.parse(xml_file).getroot()
    yolo_annotations = []

    # Get image filename
    filename = root.find('filename').text

    # Get image dimensions
    width = int(root.find('size/width').text)
    height = int(root.find('size/height').text)

    if width == 0 or height == 0:
        print(f"Error: Width or height is zero in XML file: {xml_file}")
        return filename, []

    # Calculate conversion factors
    width_ratio = image_width / width
    height_ratio = image_height / height

    # Loop through object annotations
    for obj in root.findall('object'):
        # Get object class
        obj_class = obj.find('name').text

        # Get bounding box coordinates
        xmin = int(obj.find('bndbox/xmin').text)
        ymin = int(obj.find('bndbox/ymin').text)
        xmax = int(obj.find('bndbox/xmax').text)
        ymax = int(obj.find('bndbox/ymax').text)

        # Convert bounding box coordinates to YOLO format
        x_center = (xmin + xmax) / 2.0
        y_center = (ymin + ymax) / 2.0
        box_width = xmax - xmin
        box_height = ymax - ymin

        # Normalize coordinates
        x_center /= width
        y_center /= height
        box_width /= width
        box_height /= height

        # Append YOLO format annotation to list
        yolo_annotations.append(f"{obj_class} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}")

    return filename, yolo_annotations


def write_yolo_annotations(xml_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    for xml_file in os.listdir(xml_dir):
        if xml_file.endswith('.xml'):
            xml_path = os.path.join(xml_dir, xml_file)

            # Extract image dimensions
            tree = ET.parse(xml_path)
            root = tree.getroot()
            width = int(root.find('size/width').text)
            height = int(root.find('size/height').text)

            # Convert XML to YOLO format
            image_filename, annotations = convert_xml_to_yolo(xml_path, width, height)

            # Write YOLO annotations to file
            with open(os.path.join(output_dir, f"{os.path.splitext(image_filename)[0]}.txt"), 'w') as f:
                f.write('\n'.join(annotations))

# Example usage
xml_dir = '/content/PlantDoc-Object-Detection-Dataset/TEST'  # Directory containing XML annotations
output_dir = '/content/PlantDoc-Object-Detection-Dataset/main/test/labels'  # Output directory for YOLO annotations
write_yolo_annotations(xml_dir, output_dir)


## **Removing all xml file inside the images**

In [ ]:
import os

def remove_xml_files(directory):
    # Get list of all files in the directory
    files = os.listdir(directory)

    # Iterate over each file
    for file in files:
        # Check if the file is an XML file
        if file.endswith('.xml'):
            # Construct the full path to the XML file
            file_path = os.path.join(directory, file)

            # Remove the XML file
            os.remove(file_path)

# Specify the directory containing XML files
directory = '/content/PlantDoc-Object-Detection-Dataset/TEST'

# Remove XML files
remove_xml_files(directory)


In [ ]:
import os
import shutil

# Source and destination directory paths
source_dir = '/content/PlantDoc-Object-Detection-Dataset/TEST'
destination_dir = '/content/PlantDoc-Object-Detection-Dataset/main/test/images'

# Iterate through all files and directories in the source directory
for root, dirs, files in os.walk(source_dir):
    for file in files:
        # Check if the file is a JPEG file
        if file.endswith('.jpg'):
            # Construct the source and destination paths
            source_file = os.path.join(root, file)
            dest_file = os.path.join(destination_dir, os.path.relpath(source_file, source_dir))

            # Create the destination directory if it doesn't exist
            os.makedirs(os.path.dirname(dest_file), exist_ok=True)

            # copy the file
            shutil.copy(source_file, dest_file)

print("JPEG files ccopy successfully!")


JPEG files ccopy successfully!


# **Covert main data into rar**

In [ ]:
!rar a -r PlantDoc-Object-Detection-Dataset-P.rar /content/PlantDoc-Object-Detection-Dataset/main


/bin/bash: line 1: rar: command not found


In [ ]:
!pip install ultralytics

In [ ]:
import os

# Directory containing the images
image_directory = '/content/PlantDoc-Object-Detection-Dataset/main/train/labels'

# Get the list of files in the directory
files = os.listdir(image_directory)

# Filter out only the files that have a '.jpg' extension
image_files = [file for file in files if file.endswith('txt')]

# Print the number of image files
print(f"Number of images: {len(image_files)}")


In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)

# Train the model with 2 GPUs
results = model.train(data='/content/data2.yaml', epochs=30, imgsz=640)

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/data2.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=

train: Scanning /content/train/labels.cache... 2328 images, 10 backgrounds, 0 corrupt: 100%|██████████| 2328/2328 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/test/labels.cache... 239 images, 1 backgrounds, 0 corrupt: 100%|██████████| 239/239 [00:00<?, ?it/s]


Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000294, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train2
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        239        454      0.473      0.607      0.561      0.441

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      30/30      2.43G      1.051      1.631      1.305         33        640: 100%|██████████| 146/146 [00:47<00:00,  3.10it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:02<00:00,  3.40it/s]


                   all        239        454      0.498      0.577      0.567      0.446

30 epochs completed in 0.455 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 6.3MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3011498 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:07<00:00,  1.03it/s]


                   all        239        454      0.499      0.577      0.566      0.445
       Apple Scab Leaf        239         13      0.523      0.677      0.516      0.398
            Apple leaf        239         10      0.383        0.7      0.695      0.472
       Apple rust leaf        239         11      0.607      0.727       0.62      0.478
 Bell_pepper leaf spot        239         11      0.282      0.455      0.314      0.264
      Bell_pepper leaf        239         15      0.436      0.267      0.348      0.269
        Blueberry leaf        239         22       0.41      0.364      0.368      0.259
           Cherry leaf        239         19      0.653      0.316       0.54      0.441
   Corn Gray leaf spot        239          4      0.371      0.739      0.353      0.295
      Corn leaf blight        239         12        0.7      0.667      0.819      0.714
        Corn rust leaf        239         10          1      0.897      0.931      0.759
            Peach lea

In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO('/content/runs/detect/train/weights/best.pt')

# Define path to the image file
source = '/content/test.jpg'

# Run inference on the source
results = model(source)  # list of Results objects


image 1/1 /content/test.jpg: 640x640 1 Corn leaf blight, 13.2ms
Speed: 3.3ms preprocess, 13.2ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)


In [ ]:
import zipfile
import os

# Define the folder to be zipped
folder_to_zip = '/content/runs'

# Define the name for the zip file
zip_file = '/content/runs.zip'

# Create a ZipFile object in write mode
with zipfile.ZipFile(zip_file, 'w') as zipf:
    # Iterate over the files in the folder
    for root, dirs, files in os.walk(folder_to_zip):
        for file in files:
            # Define the absolute path of the file
            file_path = os.path.join(root, file)
            # Add the file to the zip archive
            zipf.write(file_path, os.path.relpath(file_path, folder_to_zip))

# Print a message when the zip process is completed
print(f"Folder '{folder_to_zip}' zipped successfully to '{zip_file}'.")


Folder '/content/runs' zipped successfully to '/content/runs.zip'.
